#Load Dataset

In [1]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-11-11 01:45:36--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uca3ddfb87dbd000f956bdebac32.dl.dropboxusercontent.com/cd/0/inline/CeJpxYCs_IOdepPER9akNZVWTRRakTwFolIeWQioevL4Y32oUXKRYaTZ3ktzM44UOsOsGjLlG7ZH7QJqpYvyW7EFUsCI2FHHlPyhzV7pHisW_ZvwSD-b68W0YgELwTkMbVM/file?dl=1# [following]
--2024-11-11 01:45:36--  https://uca3ddfb87dbd000f956bdebac32.dl.dropboxusercontent.com/cd/0/inline/CeJpxYCs_IOdepPER9akNZVWTRRakTwFolIeWQioevL4Y32oUXKRYaTZ3ktzM44UOsOsGjLlG7ZH7QJqpYvyW7EFUsCI2FHHlPyhzV7pHisW_ZvwSD-b68W0YgELwTkMbVM/file?dl=1
Resolving uca3ddfb87dbd000f956bdebac32.dl.dropboxusercontent.com (uca3ddfb87dbd000f956bdebac32.dl.dropboxusercontent.com)... 162.125.4.15, 2620:100:601d:15::a27d

#Load HF model

In [2]:
from huggingface_hub import notebook_login

notebook_login()



In [3]:
!pip install transformers[torch]
!pip install bitsandbytes
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 15.1 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

In [5]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
def generate_prompt(data):
  question = data['question']
  ref = data['references']

  chat = [
      {"role": "user", "content": f"Answer the Question: {question} Considering this information: {ref}"},
    ]

  data['prompt'] = tokenizer.apply_chat_template(chat, tokenize=False)
  return data

#dataset['test'] = dataset['test'].map(generate_prompt)

In [ ]:
def generate_zeroshot_prompt(likes, dislikes):
  likes_str = "\n".join(likes)
  dislikes_str = "\n".join(dislikes)

  chat = [
    {"role": "user", "content": f"I've liked these movies: {likes_str}. I've disliked these movies: {dislikes_str}. Recommend a new movie for me to watch."}
  ]
  return tokenizer.apply_chat_template(chat, tokenize=False)

In [7]:
def generate_zeroshot(conversation_id, conversations):
    conversation = "\n".join(conversations[conversation_id]["messages"])
    content = f"Pretend you are a movie recommender system. I will give you a conversation between a human and assistant. Based on the conversation, you reply me with 20 recommendations without extra sentences. Here is the conversation: {conversation}."
    chat = [
        {"role": "user", "content": content}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

In [8]:
from Tools import read_user_data
path_movie_data = "/content/Movie/final_data.jsonl"

In [9]:
import json
item_map_path = "/content/Movie/item_map.json"
item_map = json.load(open(item_map_path,'r'))
user_map_path = "/content/Movie/user_ids.json"
user_map = json.load(open(user_map_path,'r'))

In [10]:
conversations_path = "/content/Movie/Conversation.txt"

conversations = []
current_conversation = []
conversation_id = 0
with open(conversations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue

        if line.isdigit():
            if current_conversation:
                conversations.append({
                    'conversation_id': conversation_id,
                    'messages': current_conversation
                })
            conversation_id = int(line)
            current_conversation = []
        else:
            current_conversation.append(line)

    if current_conversation:
        conversations.append({
            'conversation_id': conversation_id,
            'messages': current_conversation
        })

In [11]:
import random
user_ids = list(user_map.keys())
sample_percent = 0.01 # 10%

sample_size = int(sample_percent * len(user_ids))
sampled_users = random.sample(user_ids,sample_size)

In [12]:
prompts = []
ground_truth = []
for user_id in sampled_users:
  user_data = read_user_data(path_movie_data, user_id)
  conversation_id = user_data['Conversation'][0]['conversation_1']['conversation_id']
  prompt = generate_zeroshot(conversation_id, conversations)
  prompts.append(prompt)


In [15]:
answers = []
for prompt in prompts:
  ans = text_generator(prompt, max_new_tokens=300, return_full_text=False)
  answers.append(ans[0]['generated_text'])


In [16]:
for answer in answers:
  print(answer)


 Based on your interest in Trainspotting and Ichi the Killer, here are 20 movie recommendations that blend intense portrayals of addiction, violence, and dark humor:

1. Requiem for a Dream (2000)
2. A Clockwork Orange (1971)
3. Fear and Loathing in Las Vegas (1998)
4. The Wolf of Wall Street (2013)
5. American Psycho (2000)
6. Fight Club (1999)
7. The Departed (2006)
8. Gomorrah (2008)
9. The Hangover (2009)
10. The Big Short (2015)
11. The Boondock Saints (1999)
12. Sin City (2005)
13. The Dark Knight (2008)
14. The Town (2010)
15. The Raid: Redemption (2011)
16. The Kings of Summer (2013)
17. The Revenant (2015)
18. The Wolf of Snow Hollow (2020)
19. The Gentlemen (2019
 Here are 20 movie recommendations based on the user's interest in classic films, particularly sci-fi and old-time westerns:

1. "Forbidden Planet" (1956) - A classic sci-fi film about a spaceship crew investigating a mysterious planet and encountering a powerful alien creature.
2. "Invasion of the Body Snatchers" (1

In [46]:
import re

def parse_recommendations(answer):
  pattern = r'\d+\.\s+"*([^"]+?)"*(?:\s\(\d{4}\))?(?:\s*-\s*.+)?(?:\s*\(\d-\d\))?$'
  recommendations = re.findall(pattern, answer, re.MULTILINE)
  return recommendations

In [57]:
import numpy as np

def recall_at_k(r, k, m):
    return (np.asarray(r)[:k] != 0).sum() / m

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [70]:
def evaluate_model(answers, users):
    assert len(answers) == len(users)

    mean_ndcg = 0
    mean_recall = 0
    recommendations_count = 0
    for i in range(len(answers)):
       recommendations = parse_recommendations(answers[i])
       if len(recommendations) != 20:
         continue
       user_data = read_user_data(path_movie_data, users[i])
       user_might_like = [item_map[id] for id in user_data["user_might_like"]]
       rel_vector = np.isin(recommendations, user_might_like, assume_unique=True).astype(int)

       mean_ndcg += ndcg_at_k(rel_vector, 20, len(user_might_like))
       mean_recall += recall_at_k(rel_vector, 20, len(user_might_like))
       recommendations_count += 1

    mean_ndcg /= recommendations_count
    mean_recall /= recommendations_count

    return mean_ndcg, mean_recall

In [72]:
ndcg, recall = evaluate_model(answers, sampled_users)
print(f"NDCG: {ndcg:.4f}")
print(f"Recall: {recall:.4f}")

NDCG: 0.0100
Recall: 0.0156
